In [1]:
import numpy as np
import cv2
import sys, os, time, argparse, logging
import simplejson
import argparse
import math
import mylib.io as myio
from mylib.displays import drawActionResult
import mylib.funcs as myfunc
import mylib.feature_proc as myproc 
from mylib.action_classifier import ClassifierOnlineTest
from mylib.action_classifier import * # Import sklearn related libraries

In [2]:
# CURR_PATH = os.path.dirname(os.path.abspath(__file__))+"/" # ? curr_path
# DRAW_FPS = True

In [3]:
# INPUTS ============================================================== ## tạm thời bỏ qua


# def parse_input_method():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--source", required=False, default='webcam', 
#                         choices=["webcam", "folder", "txtscript"])
#     return parser.parse_args().source
 
# arg_input = parse_input_method()
# FROM_WEBCAM = arg_input == "webcam" # from web camera
# FROM_TXTSCRIPT = arg_input == "txtscript" # from a txt script (for training)
# FROM_FOLDER = arg_input == "folder" # read images from a folder

# PATHS and SETTINGS =================================

# Choose image_size from: ["640x480", "432x368", "304x240", "240x208", "208x160"]
# The higher, the better. But slower.

# if FROM_WEBCAM:
#     folder_suffix = "3"
#     DO_INFER_ACTIONS =  True
#     SAVE_RESULTANT_SKELETON_TO_TXT_AND_IMAGE = False
#     if 1: 
#         image_size = "432x368" # 10 fps
#         OpenPose_MODEL = ["mobilenet_thin", "cmu"][0]
#     else:
#         image_size = "240x208" # 10 fps
#         OpenPose_MODEL = ["mobilenet_thin", "cmu"][1]
# ua
# elif FROM_FOLDER:

# folder_suffix = "4"
# DO_INFER_ACTIONS =  True
# SAVE_RESULTANT_SKELETON_TO_TXT_AND_IMAGE = True
# def set_source_images_from_folder():
#     return "images", 0
# SRC_IMAGE_FOLDER, SKIP_NUM_IMAGES = set_source_images_from_folder()
# SKIP_NUM_IMAGES

# folder_suffix += SRC_IMAGE_FOLDER.split('/')[0] #[-2] # plus folder name
# folder_suffix

In [4]:
# image_size = "304x240"
image_size = "240x208"
OpenPose_MODEL = ["mobilenet_thin", "cmu"][1]
    

In [5]:

LOAD_MODEL_PATH = "../model/trained_classifier.pickle"
action_labels=  ['jump','kick','punch','run','sit','squat','stand','walk','wave']

    

In [6]:

# SKELETON_FOLDER = "skeleton_data/"
# SAVE_DETECTED_SKELETON_TO =         "skeleton_data/skeletons"+"/"
# SAVE_DETECTED_SKELETON_IMAGES_TO =  "skeleton_data/skeletons"+"images/"
# SAVE_IMAGES_INFO_TO =             "skeleton_data/images_info"+".txt"

In [7]:
# # create folders for saving results
# if not os.path.exists(SKELETON_FOLDER):
#     os.makedirs(SKELETON_FOLDER)
# if not os.path.exists(SAVE_DETECTED_SKELETON_TO):
#     os.makedirs(SAVE_DETECTED_SKELETON_TO)
# if not os.path.exists(SAVE_DETECTED_SKELETON_IMAGES_TO):
#     os.makedirs(SAVE_DETECTED_SKELETON_IMAGES_TO)

In [8]:
# Openpose include files and configs ==============================================================

sys.path.append("githubs/tf-pose-estimation")
from tf_pose.networks import get_graph_path, model_wh
from tf_pose.estimator import TfPoseEstimator
from tf_pose import common

W0810 11:28:03.495113 140363250124544 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/mobilenet/mobilenet.py:369: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [9]:
# logger = logging.getLogger('TfPoseEstimator')
# logger.setLevel(logging.DEBUG)
# ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
# formatter = logging.Formatter(
#     '[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
# ch.setFormatter(formatter)
# logger.addHandler(ch)

In [10]:
# ---- For tf 1.13.1, The following setting is needed
import tensorflow as tf
from tensorflow import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# If GPU memory is small, modify the MAX_FRACTION_OF_GPU_TO_USE
MAX_FRACTION_OF_GPU_TO_USE = 0.5
config.gpu_options.per_process_gpu_memory_fraction=MAX_FRACTION_OF_GPU_TO_USE

In [11]:
# Openpose Human pose detection ==============================================================

class SkeletonDetector(object):
    # This func is mostly copied from https://github.com/ildoonet/tf-pose-estimation

    def __init__(self, model=None, image_size=None):
        
        if model is None:
            model = "cmu"

        if image_size is None:
            image_size = "432x368" # 7 fps
            # image_size = "336x288"
            # image_size = "304x240" # 14 fps

        models = set({"mobilenet_thin", "cmu"})
        self.model = model if model in models else "mobilenet_thin"
        self.resize_out_ratio = 4.0

        # args = parser.parse_args()

        w, h = model_wh(image_size)
        if w == 0 or h == 0:
            e = TfPoseEstimator(
                    get_graph_path(self.model),
                    target_size=(432, 368),
                    tf_config=config)
        else:
            e = TfPoseEstimator(
                get_graph_path(self.model), 
                target_size=(w, h),
                tf_config=config)

        # self.args = args
        self.w, self.h = w, h
        self.e = e
        self.fps_time = time.time()
        self.cnt_image = 0

    def detect(self, image):
        self.cnt_image += 1
        if self.cnt_image == 1:
            self.image_h = image.shape[0]
            self.image_w = image.shape[1]
            self.scale_y = 1.0 * self.image_h / self.image_w
        t = time.time()

        # Inference
        humans = self.e.inference(image, resize_to_default=(self.w > 0 and self.h > 0),
                                #   upsample_size=self.args.resize_out_ratio)
                                  upsample_size=self.resize_out_ratio)

        # Print result and time cost
        elapsed = time.time() - t
#         logger.info('inference image in %.4f seconds.' % (elapsed))

        return humans
    
    def draw(self, img_disp, humans):
        img_disp = TfPoseEstimator.draw_humans(img_disp, humans, imgcopy=False)
        # logger.debug('show+')
#         if DRAW_FPS:
#             cv2.putText(img_disp,
#                         # "Processing speed: {:.1f} fps".format( (1.0 / (time.time() - self.fps_time) )),
#                         "fps = {:.1f}".format( (1.0 / (time.time() - self.fps_time) )),
#                         (10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 1,
#                         (0, 0, 255), 2)
#         self.fps_time = time.time()

    def humans_to_skelsList(self, humans, scale_y = None): # get (x, y * scale_y)
        # type: humans: returned from self.detect()
        # rtype: list[list[]]
        if scale_y is None:
            scale_y = self.scale_y
        skeletons = []
        NaN = 0
        for human in humans:
            skeleton = [NaN]*(18*2)
            for i, body_part in human.body_parts.items(): # iterate dict
                idx = body_part.part_idx
                skeleton[2*idx]=body_part.x
                skeleton[2*idx+1]=body_part.y * scale_y
            skeletons.append(skeleton)
        return skeletons, scale_y
    


# ==============================================================

def add_white_region_to_left_of_image(image_disp):
    r, c, d = image_disp.shape
    blank = 255 + np.zeros((r, int(c/4), d), np.uint8)
    image_disp = np.hstack((blank, image_disp))
    return image_disp

def remove_skeletons_with_few_joints(skeletons):
    good_skeletons = []
    for skeleton in skeletons:
        px = skeleton[2:2+13*2:2]
        py = skeleton[3:2+13*2:2]
        num_valid_joints = len([x for x in px if x!=0])
        num_leg_joints = len([x for x in px[-6:] if x!=0])
        total_size = max(py) - min(py)
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # IF JOINTS ARE MISSING, TRY CHANGING THESE VALUES:
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        if num_valid_joints >= 5 and total_size >= 0.1 and num_leg_joints >= 2: 
            good_skeletons.append(skeleton) # add this skeleton only when all requirements are satisfied
    return good_skeletons

class MultiPersonClassifier(object):
    def __init__(self, LOAD_MODEL_PATH, action_labels):
        self.create_classifier = lambda human_id: ClassifierOnlineTest(
            LOAD_MODEL_PATH, action_types = action_labels, human_id=human_id)
        self.dict_id2clf = {} # human id -> classifier of this person

    def classify(self, dict_id2skeleton):

        # Clear people not in view
        old_ids = set(self.dict_id2clf)
        cur_ids = set(dict_id2skeleton)
        humans_not_in_view = list(old_ids - cur_ids)
        for human in humans_not_in_view:
            del self.dict_id2clf[human]

        # Predict each person's action
        id2label = {}
        for id, skeleton in dict_id2skeleton.items():
            
            if id not in self.dict_id2clf: # add this new person
                self.dict_id2clf[id] = self.create_classifier(id)
            
            classifier = self.dict_id2clf[id]
            id2label[id] = classifier.predict(skeleton) # predict label

        return id2label

    def get(self, id):
        # type: id: int or "min"
        if len(self.dict_id2clf) == 0:
            return None 
        if id == 'min':
            id = min(self.dict_id2clf.keys())
        return self.dict_id2clf[id]

In [12]:
my_detector = SkeletonDetector(OpenPose_MODEL, image_size)

[2019-08-10 11:28:03,937] [TfPoseEstimator] [INFO] loading graph from /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose_data/graph/cmu/graph_opt.pb(default size=240x208)
I0810 11:28:03.937829 140363250124544 estimator.py:310] loading graph from /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose_data/graph/cmu/graph_opt.pb(default size=240x208)
W0810 11:28:03.942577 140363250124544 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0810 11:28:03.944469 140363250124544 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0810 11:28:04.226849 140363250124544 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/estimator.py:330: The name tf.get_default_graph is deprecated. Please use 

TfPoseEstimator/Openpose/concat_stage7/axis
TfPoseEstimator/Mconv7_stage6_L2/biases
TfPoseEstimator/Mconv7_stage6_L2/weights
TfPoseEstimator/Mconv6_stage6_L2/biases
TfPoseEstimator/Mconv6_stage6_L2/weights
TfPoseEstimator/Mconv5_stage6_L2/biases
TfPoseEstimator/Mconv5_stage6_L2/weights
TfPoseEstimator/Mconv4_stage6_L2/biases
TfPoseEstimator/Mconv4_stage6_L2/weights
TfPoseEstimator/Mconv3_stage6_L2/biases
TfPoseEstimator/Mconv3_stage6_L2/weights
TfPoseEstimator/Mconv2_stage6_L2/biases
TfPoseEstimator/Mconv2_stage6_L2/weights
TfPoseEstimator/Mconv1_stage6_L2/biases
TfPoseEstimator/Mconv1_stage6_L2/weights
TfPoseEstimator/Mconv7_stage6_L1/biases
TfPoseEstimator/Mconv7_stage6_L1/weights
TfPoseEstimator/Mconv6_stage6_L1/biases
TfPoseEstimator/Mconv6_stage6_L1/weights
TfPoseEstimator/Mconv5_stage6_L1/biases
TfPoseEstimator/Mconv5_stage6_L1/weights
TfPoseEstimator/Mconv4_stage6_L1/biases
TfPoseEstimator/Mconv4_stage6_L1/weights
TfPoseEstimator/Mconv3_stage6_L1/biases
TfPoseEstimator/Mconv3_st

In [13]:
# images_loader = myio.DataLoader_folder(SRC_IMAGE_FOLDER, SKIP_NUM_IMAGES)
# images_loader

In [14]:
# code viet
gr=[]
image =  cv2.imread('frame12.jpg')
for i in range(7):
    image_add=image
    gr.append(image_add)
# img_action_type = "unknown"
# LENGTH_OF_IMAGE_INFO = ["none"]*5
# img_info=LENGTH_OF_IMAGE_INFO
                    


In [15]:
multipeople_classifier = MultiPersonClassifier(LOAD_MODEL_PATH, action_labels)
multiperson_tracker = myfunc.Tracker()

In [18]:
for i in range(9):
    humans = my_detector.detect(image)
    print('count_human',len(humans))
    skeletons, scale_y = my_detector.humans_to_skelsList(humans)
    skeletons = remove_skeletons_with_few_joints(skeletons)
    dict_id2skeleton = multiperson_tracker.track(skeletons) # int id -> np.array() skeleton
    min_id = min(dict_id2skeleton.keys())
    dict_id2label = multipeople_classifier.classify(dict_id2skeleton)
    print("prediced label is :", dict_id2label[min_id])
    
#     my_detector.draw(image, humans) # Draw all skeletons
#     ith_img = 1
#     if len(dict_id2skeleton): 

#         # Draw outer box and label for each person 
#         for id, label in dict_id2label.items():
#             skeleton = dict_id2skeleton[id]
#             skeleton[1::2] = skeleton[1::2] / scale_y # scale the y data back to original
#             # print("Drawing skeleton: ", dict_id2skeleton[id], "with label:", label, ".")
#             drawActionResult(image, id, skeleton, label)


#     # Add blank to the left for displaying prediction scores of each class
#     image_disp = add_white_region_to_left_of_image(image)

#     # Draw predicting score for only 1 person (not using for)

#     multipeople_classifier.get(id='min').draw_scores_onto_image(image_disp)
#     image_disp


count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave
count_human 2
prediced label is : wave


In [17]:
# my_detector.draw(image, humans) # Draw all skeletons
# ith_img = 1
# if len(dict_id2skeleton): 

#     # Draw outer box and label for each person 
#     for id, label in dict_id2label.items():
#         skeleton = dict_id2skeleton[id]
#         skeleton[1::2] = skeleton[1::2] / scale_y # scale the y data back to original
#         # print("Drawing skeleton: ", dict_id2skeleton[id], "with label:", label, ".")
#         drawActionResult(image, id, skeleton, label)


# # Add blank to the left for displaying prediction scores of each class
# image_disp = add_white_region_to_left_of_image(image)

# # Draw predicting score for only 1 person (not using for)

# multipeople_classifier.get(id='min').draw_scores_onto_image(image_disp)
# image_disp

# # -- Write skeleton.txt and image.png


# # ids = sorted(dict_id2skeleton.keys())
# # skel_to_save = [img_info + dict_id2skeleton[id].tolist() for id in ids]

# # print('save')
# # myio.save_skeletons(SAVE_DETECTED_SKELETON_TO 
# #     + myfunc.int2str(ith_img, 5) + ".txt", skel_to_save)
# # cv2.imwrite(SAVE_DETECTED_SKELETON_IMAGES_TO 
# #     + myfunc.int2str(ith_img, 5) + ".png", image_disp)



In [8]:
!zip -r save.zip save_image/

  adding: save_image/ (stored 0%)
  adding: save_image/744.jpg (deflated 1%)
  adding: save_image/2745.jpg (deflated 1%)
  adding: save_image/2123.jpg (deflated 1%)
  adding: save_image/1399.jpg (deflated 1%)
  adding: save_image/2175.jpg (deflated 1%)
  adding: save_image/2657.jpg (deflated 1%)
  adding: save_image/1520.jpg (deflated 1%)
  adding: save_image/909.jpg (deflated 1%)
  adding: save_image/3817.jpg (deflated 1%)
  adding: save_image/636.jpg (deflated 1%)
  adding: save_image/2728.jpg (deflated 1%)
  adding: save_image/70.jpg (deflated 1%)
  adding: save_image/3474.jpg (deflated 1%)
  adding: save_image/3834.jpg (deflated 1%)
  adding: save_image/3019.jpg (deflated 1%)
  adding: save_image/1509.jpg (deflated 1%)
  adding: save_image/1933.jpg (deflated 1%)
  adding: save_image/1044.jpg (deflated 1%)
  adding: save_image/3050.jpg (deflated 1%)
  adding: save_image/3669.jpg (deflated 1%)
  adding: save_image/1170.jpg (deflated 1%)
  adding: save_image/3870.jpg (deflated 1%)
  a

  adding: save_image/3395.jpg (deflated 1%)
  adding: save_image/1598.jpg (deflated 1%)
  adding: save_image/2093.jpg (deflated 1%)
  adding: save_image/46.jpg (deflated 1%)
  adding: save_image/2059.jpg (deflated 1%)
  adding: save_image/3159.jpg (deflated 1%)
  adding: save_image/3971.jpg (deflated 1%)
  adding: save_image/1081.jpg (deflated 1%)
  adding: save_image/510.jpg (deflated 1%)
  adding: save_image/1402.jpg (deflated 1%)
  adding: save_image/1854.jpg (deflated 1%)
  adding: save_image/3962.jpg (deflated 1%)
  adding: save_image/3348.jpg (deflated 1%)
  adding: save_image/3709.jpg (deflated 1%)
  adding: save_image/351.jpg (deflated 1%)
  adding: save_image/2622.jpg (deflated 1%)
  adding: save_image/895.jpg (deflated 1%)
  adding: save_image/1291.jpg (deflated 1%)
  adding: save_image/3344.jpg (deflated 1%)
  adding: save_image/1982.jpg (deflated 1%)
  adding: save_image/435.jpg (deflated 1%)
  adding: save_image/3512.jpg (deflated 1%)
  adding: save_image/888.jpg (deflated

  adding: save_image/26.jpg (deflated 1%)
  adding: save_image/1716.jpg (deflated 1%)
  adding: save_image/1056.jpg (deflated 1%)
  adding: save_image/1022.jpg (deflated 1%)
  adding: save_image/3839.jpg (deflated 1%)
  adding: save_image/1906.jpg (deflated 1%)
  adding: save_image/750.jpg (deflated 1%)
  adding: save_image/3401.jpg (deflated 1%)
  adding: save_image/404.jpg (deflated 1%)
  adding: save_image/3965.jpg (deflated 1%)
  adding: save_image/3265.jpg (deflated 1%)
  adding: save_image/1065.jpg (deflated 1%)
  adding: save_image/1426.jpg (deflated 1%)
  adding: save_image/911.jpg (deflated 1%)
  adding: save_image/2944.jpg (deflated 1%)
  adding: save_image/936.jpg (deflated 1%)
  adding: save_image/1707.jpg (deflated 1%)
  adding: save_image/1329.jpg (deflated 1%)
  adding: save_image/950.jpg (deflated 1%)
  adding: save_image/1686.jpg (deflated 1%)
  adding: save_image/573.jpg (deflated 1%)
  adding: save_image/978.jpg (deflated 1%)
  adding: save_image/719.jpg (deflated 1%

  adding: save_image/3351.jpg (deflated 1%)
  adding: save_image/1909.jpg (deflated 1%)
  adding: save_image/2374.jpg (deflated 1%)
  adding: save_image/3900.jpg (deflated 1%)
  adding: save_image/863.jpg (deflated 1%)
  adding: save_image/2195.jpg (deflated 1%)
  adding: save_image/3794.jpg (deflated 1%)
  adding: save_image/3951.jpg (deflated 1%)
  adding: save_image/807.jpg (deflated 1%)
  adding: save_image/972.jpg (deflated 1%)
  adding: save_image/2706.jpg (deflated 1%)
  adding: save_image/3500.jpg (deflated 1%)
  adding: save_image/2533.jpg (deflated 0%)
  adding: save_image/2161.jpg (deflated 1%)
  adding: save_image/3166.jpg (deflated 1%)
  adding: save_image/819.jpg (deflated 1%)
  adding: save_image/2564.jpg (deflated 1%)
  adding: save_image/2932.jpg (deflated 1%)
  adding: save_image/2142.jpg (deflated 1%)
  adding: save_image/2671.jpg (deflated 1%)
  adding: save_image/2609.jpg (deflated 1%)
  adding: save_image/141.jpg (deflated 1%)
  adding: save_image/2298.jpg (deflat

  adding: save_image/3805.jpg (deflated 1%)
  adding: save_image/3104.jpg (deflated 1%)
  adding: save_image/2326.jpg (deflated 1%)
  adding: save_image/1418.jpg (deflated 1%)
  adding: save_image/3111.jpg (deflated 1%)
  adding: save_image/574.jpg (deflated 1%)
  adding: save_image/1947.jpg (deflated 1%)
  adding: save_image/1075.jpg (deflated 1%)
  adding: save_image/3969.jpg (deflated 1%)
  adding: save_image/3483.jpg (deflated 1%)
  adding: save_image/3290.jpg (deflated 1%)
  adding: save_image/1792.jpg (deflated 1%)
  adding: save_image/3491.jpg (deflated 1%)
  adding: save_image/3262.jpg (deflated 1%)
  adding: save_image/2338.jpg (deflated 1%)
  adding: save_image/1279.jpg (deflated 1%)
  adding: save_image/666.jpg (deflated 1%)
  adding: save_image/540.jpg (deflated 1%)
  adding: save_image/1325.jpg (deflated 1%)
  adding: save_image/3677.jpg (deflated 1%)
  adding: save_image/1688.jpg (deflated 1%)
  adding: save_image/1941.jpg (deflated 1%)
  adding: save_image/2803.jpg (defl

...

  adding: save_image/2091.jpg (deflated 1%)
  adding: save_image/43.jpg (deflated 1%)
  adding: save_image/933.jpg (deflated 1%)
  adding: save_image/1592.jpg (deflated 1%)
  adding: save_image/1545.jpg (deflated 1%)
  adding: save_image/1890.jpg (deflated 1%)
  adding: save_image/2514.jpg (deflated 1%)
  adding: save_image/3789.jpg (deflated 1%)
  adding: save_image/3748.jpg (deflated 1%)
  adding: save_image/419.jpg (deflated 1%)
  adding: save_image/429.jpg (deflated 1%)
  adding: save_image/1451.jpg (deflated 1%)
  adding: save_image/1995.jpg (deflated 1%)
  adding: save_image/3874.jpg (deflated 1%)
  adding: save_image/3730.jpg (deflated 1%)
  adding: save_image/2150.jpg (deflated 1%)
  adding: save_image/3443.jpg (deflated 1%)
  adding: save_image/641.jpg (deflated 1%)
  adding: save_image/3623.jpg (deflated 1%)
  adding: save_image/3309.jpg (deflated 1%)
  adding: save_image/2577.jpg (deflated 1%)
  adding: save_image/393.jpg (deflated 1%)
  adding: save_image/2312.jpg (deflated

  adding: save_image/2871.jpg (deflated 1%)
  adding: save_image/1371.jpg (deflated 1%)
  adding: save_image/323.jpg (deflated 1%)
  adding: save_image/919.jpg (deflated 1%)
  adding: save_image/254.jpg (deflated 1%)
  adding: save_image/977.jpg (deflated 1%)
  adding: save_image/2738.jpg (deflated 1%)
  adding: save_image/220.jpg (deflated 1%)
  adding: save_image/3479.jpg (deflated 1%)
  adding: save_image/299.jpg (deflated 1%)
  adding: save_image/3045.jpg (deflated 1%)
  adding: save_image/505.jpg (deflated 1%)
  adding: save_image/647.jpg (deflated 1%)
  adding: save_image/2021.jpg (deflated 1%)
  adding: save_image/3678.jpg (deflated 1%)
  adding: save_image/1401.jpg (deflated 1%)
  adding: save_image/520.jpg (deflated 1%)
  adding: save_image/3196.jpg (deflated 1%)
  adding: save_image/1017.jpg (deflated 1%)
  adding: save_image/2434.jpg (deflated 1%)
  adding: save_image/2979.jpg (deflated 1%)
  adding: save_image/3233.jpg (deflated 1%)
  adding: save_image/3009.jpg (deflated 1

  adding: save_image/3155.jpg (deflated 1%)
  adding: save_image/3137.jpg (deflated 1%)
  adding: save_image/629.jpg (deflated 1%)
  adding: save_image/3156.jpg (deflated 1%)
  adding: save_image/266.jpg (deflated 1%)
  adding: save_image/481.jpg (deflated 1%)
  adding: save_image/775.jpg (deflated 1%)
  adding: save_image/789.jpg (deflated 1%)
  adding: save_image/631.jpg (deflated 1%)
  adding: save_image/3858.jpg (deflated 1%)
  adding: save_image/3429.jpg (deflated 1%)
  adding: save_image/696.jpg (deflated 1%)
  adding: save_image/2862.jpg (deflated 1%)
  adding: save_image/1573.jpg (deflated 1%)
  adding: save_image/1868.jpg (deflated 1%)
  adding: save_image/3524.jpg (deflated 1%)
  adding: save_image/214.jpg (deflated 1%)
  adding: save_image/2245.jpg (deflated 1%)
  adding: save_image/51.jpg (deflated 1%)
  adding: save_image/2412.jpg (deflated 1%)
  adding: save_image/2143.jpg (deflated 1%)
  adding: save_image/966.jpg (deflated 1%)
  adding: save_image/2345.jpg (deflated 1%)

  adding: save_image/3618.jpg (deflated 1%)
  adding: save_image/1218.jpg (deflated 1%)
  adding: save_image/120.jpg (deflated 0%)
  adding: save_image/1631.jpg (deflated 1%)
  adding: save_image/2259.jpg (deflated 1%)
  adding: save_image/2679.jpg (deflated 1%)
  adding: save_image/2740.jpg (deflated 1%)
  adding: save_image/2669.jpg (deflated 1%)
  adding: save_image/3865.jpg (deflated 1%)
  adding: save_image/2162.jpg (deflated 1%)
  adding: save_image/1038.jpg (deflated 1%)
  adding: save_image/3267.jpg (deflated 1%)
  adding: save_image/746.jpg (deflated 1%)
  adding: save_image/3797.jpg (deflated 1%)
  adding: save_image/2692.jpg (deflated 1%)
  adding: save_image/1080.jpg (deflated 1%)
  adding: save_image/1142.jpg (deflated 1%)
  adding: save_image/2218.jpg (deflated 1%)
  adding: save_image/2399.jpg (deflated 1%)
  adding: save_image/1823.jpg (deflated 1%)
  adding: save_image/1624.jpg (deflated 1%)
  adding: save_image/3815.jpg (deflated 1%)
  adding: save_image/2844.jpg (def

  adding: save_image/655.jpg (deflated 1%)
  adding: save_image/3473.jpg (deflated 1%)
  adding: save_image/1939.jpg (deflated 1%)
  adding: save_image/1176.jpg (deflated 1%)
  adding: save_image/3706.jpg (deflated 1%)
  adding: save_image/2464.jpg (deflated 1%)
  adding: save_image/3919.jpg (deflated 1%)
  adding: save_image/3243.jpg (deflated 1%)
  adding: save_image/1.jpg (deflated 1%)
  adding: save_image/23.jpg (deflated 1%)
  adding: save_image/830.jpg (deflated 1%)
  adding: save_image/1304.jpg (deflated 1%)
  adding: save_image/1110.jpg (deflated 1%)
  adding: save_image/3660.jpg (deflated 1%)
  adding: save_image/84.jpg (deflated 1%)
  adding: save_image/2222.jpg (deflated 1%)
  adding: save_image/1050.jpg (deflated 1%)
  adding: save_image/916.jpg (deflated 1%)
  adding: save_image/3461.jpg (deflated 1%)
  adding: save_image/3768.jpg (deflated 1%)
  adding: save_image/1619.jpg (deflated 1%)
  adding: save_image/1680.jpg (deflated 1%)
  adding: save_image/3614.jpg (deflated 1%

In [6]:
!rm -rf save_image

In [7]:
!unzip frame_train2.zip

Archive:  frame_train2.zip
   creating: frame_train2/
  inflating: frame_train2/frame100.jpg  
  inflating: frame_train2/frame1000.jpg  
  inflating: frame_train2/frame1001.jpg  
  inflating: frame_train2/frame1002.jpg  
  inflating: frame_train2/frame1003.jpg  
  inflating: frame_train2/frame1004.jpg  
  inflating: frame_train2/frame1005.jpg  
  inflating: frame_train2/frame1006.jpg  
  inflating: frame_train2/frame1007.jpg  
  inflating: frame_train2/frame1008.jpg  
  inflating: frame_train2/frame1009.jpg  
  inflating: frame_train2/frame101.jpg  
  inflating: frame_train2/frame1010.jpg  
  inflating: frame_train2/frame1011.jpg  
  inflating: frame_train2/frame1012.jpg  
  inflating: frame_train2/frame1013.jpg  
  inflating: frame_train2/frame1014.jpg  
  inflating: frame_train2/frame1015.jpg  
  inflating: frame_train2/frame1016.jpg  
  inflating: frame_train2/frame1017.jpg  
  inflating: frame_train2/frame1018.jpg  
  inflating: frame_train2/frame1019.jpg  
  inflating: frame_train

  inflating: frame_train2/frame1238.jpg  
  inflating: frame_train2/frame1239.jpg  
  inflating: frame_train2/frame124.jpg  
  inflating: frame_train2/frame1240.jpg  
  inflating: frame_train2/frame1241.jpg  
  inflating: frame_train2/frame1242.jpg  
  inflating: frame_train2/frame1243.jpg  
  inflating: frame_train2/frame1244.jpg  
  inflating: frame_train2/frame1245.jpg  
  inflating: frame_train2/frame1246.jpg  
  inflating: frame_train2/frame1247.jpg  
  inflating: frame_train2/frame1248.jpg  
  inflating: frame_train2/frame1249.jpg  
  inflating: frame_train2/frame125.jpg  
  inflating: frame_train2/frame1250.jpg  
  inflating: frame_train2/frame1251.jpg  
  inflating: frame_train2/frame1252.jpg  
  inflating: frame_train2/frame1253.jpg  
  inflating: frame_train2/frame1254.jpg  
  inflating: frame_train2/frame1255.jpg  
  inflating: frame_train2/frame1256.jpg  
  inflating: frame_train2/frame1257.jpg  
  inflating: frame_train2/frame1258.jpg  
  inflating: frame_train2/frame1259.

  inflating: frame_train2/frame1427.jpg  
  inflating: frame_train2/frame1428.jpg  
  inflating: frame_train2/frame1429.jpg  
  inflating: frame_train2/frame143.jpg  
  inflating: frame_train2/frame1430.jpg  
  inflating: frame_train2/frame1431.jpg  
  inflating: frame_train2/frame1432.jpg  
  inflating: frame_train2/frame1433.jpg  
  inflating: frame_train2/frame1434.jpg  
  inflating: frame_train2/frame1435.jpg  
  inflating: frame_train2/frame1436.jpg  
  inflating: frame_train2/frame1437.jpg  
  inflating: frame_train2/frame1438.jpg  
  inflating: frame_train2/frame1439.jpg  
  inflating: frame_train2/frame144.jpg  
  inflating: frame_train2/frame1440.jpg  
  inflating: frame_train2/frame1441.jpg  
  inflating: frame_train2/frame1442.jpg  
  inflating: frame_train2/frame1443.jpg  
  inflating: frame_train2/frame1444.jpg  
  inflating: frame_train2/frame1445.jpg  
  inflating: frame_train2/frame1446.jpg  
  inflating: frame_train2/frame1447.jpg  
  inflating: frame_train2/frame1448.

  inflating: frame_train2/frame1749.jpg  
  inflating: frame_train2/frame175.jpg  
  inflating: frame_train2/frame1750.jpg  
  inflating: frame_train2/frame1751.jpg  
  inflating: frame_train2/frame1752.jpg  
  inflating: frame_train2/frame1753.jpg  
  inflating: frame_train2/frame1754.jpg  
  inflating: frame_train2/frame1755.jpg  
  inflating: frame_train2/frame1756.jpg  
  inflating: frame_train2/frame1757.jpg  
  inflating: frame_train2/frame1758.jpg  
  inflating: frame_train2/frame1759.jpg  
  inflating: frame_train2/frame176.jpg  
  inflating: frame_train2/frame1760.jpg  
  inflating: frame_train2/frame1761.jpg  
  inflating: frame_train2/frame1762.jpg  
  inflating: frame_train2/frame1763.jpg  
  inflating: frame_train2/frame1764.jpg  
  inflating: frame_train2/frame1765.jpg  
  inflating: frame_train2/frame1766.jpg  
  inflating: frame_train2/frame1767.jpg  
  inflating: frame_train2/frame1768.jpg  
  inflating: frame_train2/frame1769.jpg  
  inflating: frame_train2/frame177.j

  inflating: frame_train2/frame1943.jpg  
  inflating: frame_train2/frame1944.jpg  
  inflating: frame_train2/frame1945.jpg  
  inflating: frame_train2/frame1946.jpg  
  inflating: frame_train2/frame1947.jpg  
  inflating: frame_train2/frame1948.jpg  
  inflating: frame_train2/frame1949.jpg  
  inflating: frame_train2/frame195.jpg  
  inflating: frame_train2/frame1950.jpg  
  inflating: frame_train2/frame1951.jpg  
  inflating: frame_train2/frame1952.jpg  
  inflating: frame_train2/frame1953.jpg  
  inflating: frame_train2/frame1954.jpg  
  inflating: frame_train2/frame1955.jpg  
  inflating: frame_train2/frame1956.jpg  
  inflating: frame_train2/frame1957.jpg  
  inflating: frame_train2/frame1958.jpg  
  inflating: frame_train2/frame1959.jpg  
  inflating: frame_train2/frame196.jpg  
  inflating: frame_train2/frame1960.jpg  
  inflating: frame_train2/frame1961.jpg  
  inflating: frame_train2/frame1962.jpg  
  inflating: frame_train2/frame1963.jpg  
  inflating: frame_train2/frame1964.

...

  inflating: frame_train2/frame5975.jpg  
  inflating: frame_train2/frame5976.jpg  
  inflating: frame_train2/frame5977.jpg  
  inflating: frame_train2/frame5978.jpg  
  inflating: frame_train2/frame5979.jpg  
  inflating: frame_train2/frame5980.jpg  
  inflating: frame_train2/frame5981.jpg  
  inflating: frame_train2/frame5982.jpg  
  inflating: frame_train2/frame5983.jpg  
  inflating: frame_train2/frame5984.jpg  
  inflating: frame_train2/frame5985.jpg  
  inflating: frame_train2/frame5986.jpg  
  inflating: frame_train2/frame5987.jpg  
  inflating: frame_train2/frame5988.jpg  
  inflating: frame_train2/frame5989.jpg  
  inflating: frame_train2/frame5990.jpg  
  inflating: frame_train2/frame5991.jpg  
  inflating: frame_train2/frame5992.jpg  
  inflating: frame_train2/frame5993.jpg  
  inflating: frame_train2/frame5994.jpg  
  inflating: frame_train2/frame5995.jpg  
  inflating: frame_train2/frame5996.jpg  
  inflating: frame_train2/frame5997.jpg  
  inflating: frame_train2/frame599

  inflating: frame_train2/frame6657.jpg  
  inflating: frame_train2/frame6658.jpg  
  inflating: frame_train2/frame6659.jpg  
  inflating: frame_train2/frame6660.jpg  
  inflating: frame_train2/frame6661.jpg  
  inflating: frame_train2/frame6662.jpg  
  inflating: frame_train2/frame6663.jpg  
  inflating: frame_train2/frame6664.jpg  
  inflating: frame_train2/frame6665.jpg  
  inflating: frame_train2/frame6666.jpg  
  inflating: frame_train2/frame6667.jpg  
  inflating: frame_train2/frame6668.jpg  
  inflating: frame_train2/frame6669.jpg  
  inflating: frame_train2/frame6670.jpg  
  inflating: frame_train2/frame6671.jpg  
  inflating: frame_train2/frame6672.jpg  
  inflating: frame_train2/frame6673.jpg  
  inflating: frame_train2/frame6674.jpg  
  inflating: frame_train2/frame6675.jpg  
  inflating: frame_train2/frame6676.jpg  
  inflating: frame_train2/frame6677.jpg  
  inflating: frame_train2/frame6678.jpg  
  inflating: frame_train2/frame6679.jpg  
  inflating: frame_train2/frame668

  inflating: frame_train2/frame6857.jpg  
  inflating: frame_train2/frame6858.jpg  
  inflating: frame_train2/frame6859.jpg  
  inflating: frame_train2/frame6860.jpg  
  inflating: frame_train2/frame6861.jpg  
  inflating: frame_train2/frame6862.jpg  
  inflating: frame_train2/frame6863.jpg  
  inflating: frame_train2/frame6864.jpg  
  inflating: frame_train2/frame6865.jpg  
  inflating: frame_train2/frame6866.jpg  
  inflating: frame_train2/frame6867.jpg  
  inflating: frame_train2/frame6868.jpg  
  inflating: frame_train2/frame6869.jpg  
  inflating: frame_train2/frame6870.jpg  
  inflating: frame_train2/frame6871.jpg  
  inflating: frame_train2/frame6872.jpg  
  inflating: frame_train2/frame6873.jpg  
  inflating: frame_train2/frame6874.jpg  
  inflating: frame_train2/frame6875.jpg  
  inflating: frame_train2/frame6876.jpg  
  inflating: frame_train2/frame6877.jpg  
  inflating: frame_train2/frame6878.jpg  
  inflating: frame_train2/frame6879.jpg  
  inflating: frame_train2/frame688

  inflating: frame_train2/frame793.jpg  
  inflating: frame_train2/frame794.jpg  
  inflating: frame_train2/frame795.jpg  
  inflating: frame_train2/frame796.jpg  
  inflating: frame_train2/frame797.jpg  
  inflating: frame_train2/frame798.jpg  
  inflating: frame_train2/frame799.jpg  
  inflating: frame_train2/frame80.jpg  
  inflating: frame_train2/frame800.jpg  
  inflating: frame_train2/frame801.jpg  
  inflating: frame_train2/frame802.jpg  
  inflating: frame_train2/frame803.jpg  
  inflating: frame_train2/frame804.jpg  
  inflating: frame_train2/frame805.jpg  
  inflating: frame_train2/frame806.jpg  
  inflating: frame_train2/frame807.jpg  
  inflating: frame_train2/frame808.jpg  
  inflating: frame_train2/frame809.jpg  
  inflating: frame_train2/frame81.jpg  
  inflating: frame_train2/frame810.jpg  
  inflating: frame_train2/frame811.jpg  
  inflating: frame_train2/frame812.jpg  
  inflating: frame_train2/frame813.jpg  
  inflating: frame_train2/frame814.jpg  
  inflating: frame

  inflating: frame_train2/frame986.jpg  
  inflating: frame_train2/frame987.jpg  
  inflating: frame_train2/frame988.jpg  
  inflating: frame_train2/frame989.jpg  
  inflating: frame_train2/frame99.jpg  
  inflating: frame_train2/frame990.jpg  
  inflating: frame_train2/frame991.jpg  
  inflating: frame_train2/frame992.jpg  
  inflating: frame_train2/frame993.jpg  
  inflating: frame_train2/frame994.jpg  
  inflating: frame_train2/frame995.jpg  
  inflating: frame_train2/frame996.jpg  
  inflating: frame_train2/frame997.jpg  
  inflating: frame_train2/frame998.jpg  
  inflating: frame_train2/frame999.jpg  
